In [1]:
import sparknlp
spark = sparknlp.start() 
# sparknlp.start(gpu=True) >> for training on GPUfrom sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline
import pandas as pd
print("Spark NLP version", sparknlp.version())
print("Apache Spark version:", spark.version)

23/02/23 22:30:55 WARN Utils: Your hostname, Vader resolves to a loopback address: 127.0.1.1; using 192.168.1.94 instead (on interface enp5s0f0)
23/02/23 22:30:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/sjhjrok/anaconda3/envs/NLP/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sjhjrok/.ivy2/cache
The jars for the packages stored in: /home/sjhjrok/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-98ad61ce-e8aa-4d02-9592-fa8177f06487;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.projectlombok#l

23/02/23 22:30:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/23 22:30:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
Spark NLP version 4.2.2
Apache Spark version: 3.3.1


In [2]:
#! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_train.csv

In [3]:
#! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_test.csv

In [4]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("text_train.csv")
trainDataset.show(10, truncate=50)

+--------------------------------+--------+
|                     description|category|
+--------------------------------+--------+
|           i want to make coffee|      DA|
|                      where am i|      LR|
|              what can i do here|      AR|
|                    prepare meal|      DA|
|                use refrigerator|      DA|
|                         use fan|      DA|
|                        use oven|      DA|
|                       use stove|      DA|
|     i would like to wash sheets|      DA|
|i would like to watch television|      DA|
+--------------------------------+--------+
only showing top 10 rows



In [5]:
testDataset = spark.read \
      .option("header", True) \
      .csv("text_test.csv")
testDataset.show(10, truncate=50)

+--------------------------------+--------+
|                     description|category|
+--------------------------------+--------+
|           i want to make coffee|      DA|
|                      where am i|      LR|
|              what can i do here|      AR|
|                    prepare meal|      DA|
|                use refrigerator|      DA|
|                         use fan|      DA|
|                        use oven|      DA|
|                       use stove|      DA|
|     i would like to wash sheets|      DA|
|i would like to watch television|      DA|
+--------------------------------+--------+
only showing top 10 rows



In [6]:
from pyspark.sql.functions import col
trainDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|category|count|
+--------+-----+
|      DA|   29|
|      CC|   26|
|      SD|   23|
|      LR|   21|
|      SN|   20|
|      AR|   20|
|     NAV|   18|
|      FF|   13|
|      SV|   12|
|      RV|   11|
|      PV|   10|
|      NA|    1|
+--------+-----+



In [7]:
testDataset = spark.read \
      .option("header", True) \
      .csv("text_test.csv")
testDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|category|count|
+--------+-----+
|      DA|   29|
|      CC|   26|
|      SD|   23|
|      LR|   21|
|      SN|   20|
|      AR|   20|
|     NAV|   18|
|      FF|   13|
|      SV|   12|
|      RV|   11|
|      PV|   10|
|      NA|    1|
+--------+-----+



In [8]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")
    
# we can also use sentence detector here 
# if we want to train on and get predictions for each sentence# downloading pretrained embeddings
use = UniversalSentenceEncoder.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")
# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("category")\
    .setMaxEpochs(500)\
    .setLr(0.003)\
    .setBatchSize(32)\
    .setRandomSeed(0)\
    .setVerbose(1)\
    .setValidationSplit(0.2)\
    .setEvaluationLogExtended(True) \
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('classifer_logs')\

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

use_pipelineModel = use_clf_pipeline.fit(trainDataset)


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ]tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ / ]Download done! Loading the resource.
[ | ]

2023-02-23 22:31:12.259083: I external/org_tensorflow/tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[OK!]


2023-02-23 22:31:21.010621: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/2bd00b95f42e_classifier_dl1399580444602019389
2023-02-23 22:31:21.038971: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2023-02-23 22:31:21.038992: I external/org_tensorflow/tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /tmp/2bd00b95f42e_classifier_dl1399580444602019389
2023-02-23 22:31:21.296501: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-02-23 22:31:21.694193: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/2bd00b95f42e_classifier_dl1399580444602019389
2023-02-23 22:31:21.789533: I external/org_tensorflow/tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 778922 microsecond

Training started - epochs: 500 - learning_rate: 0.003 - batch_size: 32 - training_examples: 164 - classes: 12
Epoch 1/500 - 0.20s - loss: 14.3809805 - acc: 0.2125 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.15s
label        tp	 fp	 fn	 prec	 rec	 f1
SV           0	 0	 3	 0.0	 0.0	 0.0
DA           7	 5	 2	 0.5833333	 0.7777778	 0.6666666
NAV          0	 0	 3	 0.0	 0.0	 0.0
SD           0	 0	 5	 0.0	 0.0	 0.0
RV           0	 0	 1	 0.0	 0.0	 0.0
CC           7	 21	 0	 0.25	 1.0	 0.4
AR           0	 0	 6	 0.0	 0.0	 0.0
SN           0	 0	 3	 0.0	 0.0	 0.0
LR           0	 0	 3	 0.0	 0.0	 0.0
tp: 14 fp: 26 fn: 26 labels: 9
Macro-average	 prec: 0.09259259, rec: 0.19753087, f1: 0.12608352
Micro-average	 prec: 0.35, recall: 0.35, f1: 0.35
Epoch 2/500 - 0.02s - loss: 13.848122 - acc: 0.3875 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 

...

time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
SV           0	 0	 3	 0.0	 0.0	 0.0
DA           9	 1	 0	 0.9	 1.0	 0.9473684
NAV          2	 0	 1	 1.0	 0.6666667	 0.8
SD           3	 1	 2	 0.75	 0.6	 0.6666667
RV           0	 0	 1	 0.0	 0.0	 0.0
PV           0	 3	 0	 0.0	 0.0	 0.0
CC           7	 1	 0	 0.875	 1.0	 0.93333334
AR           5	 1	 1	 0.8333333	 0.8333333	 0.8333333
SN           3	 0	 0	 1.0	 1.0	 1.0
LR           2	 2	 1	 0.5	 0.6666667	 0.57142854
tp: 31 fp: 9 fn: 9 labels: 10
Macro-average	 prec: 0.5858334, rec: 0.5766667, f1: 0.5812139
Micro-average	 prec: 0.775, recall: 0.775, f1: 0.775
Epoch 440/500 - 0.02s - loss: 11.612888 - acc: 1.0 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
SV           0	 0	 3	 0.0	 0.0	 0.0
DA           9	 1	 0	 0.9	 1.0	 0.9473684
NAV          2	 0	 1	 1.0	 0.6666667	 0.8
SD           3	 1	 2	 0.75	 0.6	 0.6666667
R

Epoch 452/500 - 0.01s - loss: 11.610052 - acc: 1.0 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
SV           0	 0	 3	 0.0	 0.0	 0.0
DA           9	 1	 0	 0.9	 1.0	 0.9473684
NAV          2	 0	 1	 1.0	 0.6666667	 0.8
SD           3	 1	 2	 0.75	 0.6	 0.6666667
RV           0	 0	 1	 0.0	 0.0	 0.0
PV           0	 3	 0	 0.0	 0.0	 0.0
CC           7	 1	 0	 0.875	 1.0	 0.93333334
AR           5	 1	 1	 0.8333333	 0.8333333	 0.8333333
SN           3	 0	 0	 1.0	 1.0	 1.0
LR           2	 2	 1	 0.5	 0.6666667	 0.57142854
tp: 31 fp: 9 fn: 9 labels: 10
Macro-average	 prec: 0.5858334, rec: 0.5766667, f1: 0.5812139
Micro-average	 prec: 0.775, recall: 0.775, f1: 0.775
Epoch 453/500 - 0.01s - loss: 11.609819 - acc: 1.0 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
SV           0	 0	 3	 0.0	 0.0	 0.0


Epoch 466/500 - 0.01s - loss: 11.606795 - acc: 1.0 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
SV           0	 0	 3	 0.0	 0.0	 0.0
DA           9	 1	 0	 0.9	 1.0	 0.9473684
NAV          2	 0	 1	 1.0	 0.6666667	 0.8
SD           3	 1	 2	 0.75	 0.6	 0.6666667
RV           0	 0	 1	 0.0	 0.0	 0.0
PV           0	 3	 0	 0.0	 0.0	 0.0
CC           7	 1	 0	 0.875	 1.0	 0.93333334
AR           5	 1	 1	 0.8333333	 0.8333333	 0.8333333
SN           3	 0	 0	 1.0	 1.0	 1.0
LR           2	 2	 1	 0.5	 0.6666667	 0.57142854
tp: 31 fp: 9 fn: 9 labels: 10
Macro-average	 prec: 0.5858334, rec: 0.5766667, f1: 0.5812139
Micro-average	 prec: 0.775, recall: 0.775, f1: 0.775
Epoch 467/500 - 0.01s - loss: 11.6065645 - acc: 1.0 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
SV           0	 0	 3	 0.0	 0.0	 0.0

time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
SV           0	 0	 3	 0.0	 0.0	 0.0
DA           9	 1	 0	 0.9	 1.0	 0.9473684
NAV          2	 0	 1	 1.0	 0.6666667	 0.8
SD           3	 1	 2	 0.75	 0.6	 0.6666667
RV           0	 0	 1	 0.0	 0.0	 0.0
PV           0	 3	 0	 0.0	 0.0	 0.0
CC           7	 1	 0	 0.875	 1.0	 0.93333334
AR           5	 1	 1	 0.8333333	 0.8333333	 0.8333333
SN           3	 0	 0	 1.0	 1.0	 1.0
LR           2	 2	 1	 0.5	 0.6666667	 0.57142854
tp: 31 fp: 9 fn: 9 labels: 10
Macro-average	 prec: 0.5858334, rec: 0.5766667, f1: 0.5812139
Micro-average	 prec: 0.775, recall: 0.775, f1: 0.775
Epoch 480/500 - 0.01s - loss: 11.603597 - acc: 1.0 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
SV           0	 0	 3	 0.0	 0.0	 0.0
DA           9	 1	 0	 0.9	 1.0	 0.9473684
NAV          2	 0	 1	 1.0	 0.6666667	 0.8
SD           3	 1	 2	 0.75	 0.6	 0.6666667
R

time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
SV           0	 0	 3	 0.0	 0.0	 0.0
DA           9	 1	 0	 0.9	 1.0	 0.9473684
NAV          2	 0	 1	 1.0	 0.6666667	 0.8
SD           3	 1	 2	 0.75	 0.6	 0.6666667
RV           0	 0	 1	 0.0	 0.0	 0.0
PV           0	 3	 0	 0.0	 0.0	 0.0
CC           7	 1	 0	 0.875	 1.0	 0.93333334
AR           5	 1	 1	 0.8333333	 0.8333333	 0.8333333
SN           3	 0	 0	 1.0	 1.0	 1.0
LR           2	 2	 1	 0.5	 0.6666667	 0.57142854
tp: 31 fp: 9 fn: 9 labels: 10
Macro-average	 prec: 0.5858334, rec: 0.5766667, f1: 0.5812139
Micro-average	 prec: 0.775, recall: 0.775, f1: 0.775
Epoch 492/500 - 0.01s - loss: 11.600909 - acc: 1.0 - batches: 6
Quality on validation dataset (20.0%), validation examples = 40
time to finish evaluation: 0.00s
label        tp	 fp	 fn	 prec	 rec	 f1
SV           0	 0	 3	 0.0	 0.0	 0.0
DA           9	 1	 0	 0.9	 1.0	 0.9473684
NAV          2	 0	 1	 1.0	 0.6666667	 0.8
SD           3	 1	 2	 0.75	 0.6	 0.6666667
R

In [9]:
use_pipelineModel.stages[2].write().overwrite().save('Text_Classification')

In [10]:
#predictions =  use_pipelineModel.transform(testDataset)
#predictions.select("category", "text", "class.result").show(5, truncate=30)

In [11]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")
    
# we can also use sentence detector here 
# if we want to train on and get predictions for each sentence# downloading pretrained embeddings
use = UniversalSentenceEncoder.pretrained()\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")
# the classes/labels/categories are in category column
loaded_ner_model = ClassifierDLModel.load("Text_Classification")\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        loaded_ner_model
    ])

#use_pipelineModel = use_clf_pipeline.fit(trainDataset)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [12]:
from sparknlp.base import LightPipeline
#clf_pipelineModel = use_clf_pipeline.fit(trainDataset)

text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = use_clf_pipeline.fit(prediction_data)
light = LightPipeline(prediction_model)
results = light.annotate(text)
print(results)
print(results['class'])

{'document': ['navigate to kitchen'], 'sentence_embeddings': ['navigate to kitchen'], 'class': ['NAV']}
['NAV']


In [13]:
text = input("Enter Testing Text\n")
while(text != "exit"):
    results = light.annotate(text)
    print(results['class'])
    text = input("Enter New Text\n")

Enter Testing Text
stop camera
['CC']
Enter New Text
stop video
['PV']
Enter New Text
start video
['FF']
Enter New Text
skip video
['FF']
Enter New Text
play
['FF']
Enter New Text
quit
['SD']
Enter New Text
exit
